In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import json
from pathlib import Path
from datetime import timedelta, datetime

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from wikipedia_cleanup.data_processing import read_file, get_data
from wikipedia_cleanup.baseline import next_change

In [3]:
mp_drive_dir = Path("/media/hpi_share/")#Path("//FS23/projekte$/MP2021/MPWS2021/MPWS2021FN1")
mp_plot_dir = mp_drive_dir / "plots"
input_path = Path("../../data")
input_data = list(input_path.rglob("*.json"))
files = [x for x in input_data if x.is_file()]
len(files) # total 580

586

In [4]:
data = get_data(input_path, n_files=3, n_jobs=6)

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
data['validFrom'] = data['validFrom'].dt.floor('d').dt.tz_localize(None)
data['validTo'] =  data['validTo'].dt.tz_localize(None)

In [6]:
train_data = data[~data['validTo'].isna()]

# Predict the last timestamp

In [70]:
timestamps_per_page_property = train_data.groupby(['pageID', 'property.name'])['validFrom']
labels = []
predicted_values = []
for key, values in tqdm(timestamps_per_page_property):
    if len(values) > 2:
        to_predict = sorted(values)[-1]
        labels.append(to_predict)
        predicted = next_change(values)
        predicted_values.append(predicted)

  0%|          | 0/187960 [00:00<?, ?it/s]

In [71]:
labels = np.array(labels)
predicted_values = np.array(predicted_values)
no_pred_filter = predicted_values!= None

In [72]:
r = np.abs(predicted_values[no_pred_filter] - labels.astype(np.datetime64)[no_pred_filter])
converted_error = r.astype('timedelta64[D]').astype(np.int)
mean_absolute_error = np.mean(converted_error)
root_squared_mean_error = np.sqrt(np.mean(np.square(converted_error)))
print(f"Mean days absolute error: {mean_absolute_error}") 
print(f"Root squared mean error: {root_squared_mean_error}") 
# With unique
#Mean
#Mean days absolute error: 481.6636837864201
#Root squared mean error: 704.0731080458512

# Median
#Mean days absolute error: 414.5928156410365
#Root squared mean error: 669.8950754439018

#With Sort

# Median
#Mean days absolute error: 224.4591567177933
#Root squared mean error: 421.5340919090345

# Mean
#Mean days absolute error: 325.01978170499456
#Root squared mean error: 482.0884751074065

#Median unique: 374.2281070831378

Mean days absolute error: 325.01978170499456
Root squared mean error: 482.0884751074065


In [58]:
test_data = data["validFrom"] > pd.datetime(2018, 9, 1)

NameError: name 'data' is not defined

In [52]:
x["validFrom"] = x["validFrom"].dt.tz_localize(None)

# Is the value changing next year?

In [ ]:
is_test_data = (data["validFrom"] > datetime(2018, 9, 1)) & (data["validFrom"] < datetime(2019, 9, 1))
is_train_data = data["validFrom"] <= datetime(2018, 9, 1)
train_data = data[is_train_data]
test_data = data[is_test_data]
test_data = test_data.set_index(['pageID', 'property.name'])
#test_data[test_data.set_index(['pageID', 'property.name']).index.isin(
#    train_data.set_index(['pageID', 'property.name']).index)]

In [110]:
train_data['next_year'] = (train_data["validTo"] > datetime(2018, 9, 1)) & (data["validTo"] < datetime(2019, 9, 1))

/home/niklaskoehnecke/project/wikipedia_cleanup/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [111]:
train_data

,key,revisionId,pageID,pageTitle,property.name,previousValue,currentValue,validFrom,validTo,next_year
0,119505991-0,119505991,26531,Roland Corporation,revenue,None,,2007-04-01,2017-10-17 17:02:02,False
1,119505991-0,119505991,26531,Roland Corporation,homepage,None,http://www.roland.com/,2007-04-01,2007-09-12 19:37:04,False
2,119505991-0,119505991,26531,Roland Corporation,operating_income,None,,2007-04-01,2017-10-17 17:02:02,False
3,119505991-0,119505991,26531,Roland Corporation,industry,None,Conglomerate,2007-04-01,2007-04-18 02:18:38,False
4,119505991-0,119505991,26531,Roland Corporation,location_city,None,"[[Osaka, Japan|Osaka]]",2007-04-01,2007-04-18 02:18:38,False
...,...,...,...,...,...,...,...,...,...,...
2067634,389129013-0,685083433,2535092,Kaveh Golestan,birth_date,"July 8, 1950",8 July 1950,2015-10-10,NaT,False
2067635,389129013-0,764965408,2535092,Kaveh Golestan,family,[[Ebrahim Golestan]]<br />[[Lili Golestan]]<br...,[[Ebrahim Golestan]]<br />[[Lili Golestan]]<br...,2017-02-11,2019-03-14 02:26:08,True
2067636,389129013-0,775242979,2535092,Kaveh Golestan,name,Kaveh Golsetan<br>کاوه گلستان,Kaveh Golestan<br>کاوه گلستان,2017-04-13,2019-03-14 02:26:58,True
2067637,389129013-0,833859578,2535092,Kaveh Golestan,education,[[Millfield]]<ref>http://www.telegraph.co.uk/n...,[[Millfield]]<ref>https://www.telegraph.co.uk/...,2018-04-02,2018-07-09 13:21:49,False
